# VWAP - Tick Forecast

# Debug: update quantity_behind after placing order aggressively
Set tick factor coefficient to 1. Add some logs to the algo_loop, we find that at bar $9$, after placing order aggressively, the quantity_behind will not update, which causes the endless aggressive trading until the total shares are sold out. Thus, we add a line to update the quantity_behind to eliminate the bug.

During the class, professor set the tick factor to 0 and displayed that it would perfectly follow the VWAP. The reason is if we ignore the tick factor, we will never place order aggressively in our loop (the fair value is always above bid_price), given the specific TSLA TAQ data set.
# Exact number of shares traded
After fixing bug, the total shares traded is slightly less than our target $30000$. The reason is at the last bar we trade passively, and our orders are not guaranteed to be fufilled. In order to finish all the orders, at the last bar, we force it to trade aggressively, which achieve our target more precisely but slightly increase the trading cost.

Another thing that prevents us from trading exact $30000$ shares is that we can only trade mutiple of 100 shares when trading aggressively (round_lot=100). We can not change this because it's determined by the exchange. 

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# load our helper functios
import simtools as st
import vwap2 as vw

matplotlib.rcParams[ 'figure.figsize' ] = ( 16, 9 )

In [2]:
vwap_coefs = [ 9.95503275e-03, -1.01533564e-04,  5.73879785e-07, -1.47027197e-09, 1.38993793e-12]

In [3]:
# set up some variables
tickfile = 'TSLA-TAQ-jan20.zip'
test_day = '2020-01-15'
taq = pd.read_pickle(tickfile)
# filter on date
day = taq[test_day]

In [4]:
# without the tick signal...
order_quantity = 300000
order_side = 's'
results = vw.algo_loop(day, order_side, order_quantity, vwap_coefs, )

TypeError: algo_loop() missing 1 required positional argument: 'schedule_coef'

In [ ]:
results['fair_values'].plot()

In [ ]:
results['midpoints'].plot()
day.trade_px.plot()

In [ ]:
data = results['trades']

In [ ]:
data.price.plot()

In [ ]:
data['shares'].sum()

In [ ]:
results.keys()

In [ ]:
results['day_vwap']

In [ ]:
results['avg_price']

In [ ]:
#slippage vs. vwap
results['day_vwap'] - results['avg_price']

In [ ]:
results['trades'].groupby(['trade_type']).count()

In [ ]:
results['trades']

In [ ]:
[print(results['schedule_factors'][i]) for i in range(results['schedule_factors'].shape[0])]

## Pick the best Schedule

The TAQ data contain time range 2020/01/15 - 2020/01/17. Above we just test the day 2020/01/15.

In [6]:
scheduel_coefs = np.linspace(0.01, 0.2, 20)[::-1]
if order_side == 's':
    scheduel_coefs = -scheduel_coefs
for i, schedule_coef in enumerate(scheduel_coefs):
    results = vw.algo_loop(day, order_side, order_quantity, vwap_coefs, schedule_coef)
    print(results['day_vwap'] - results['avg_price'])
    print(results['trades'].groupby(['trade_type']).count())

2020-03-21 18:23:48:278952: Beginning VWAP run: s 300000 shares
2020-03-21 18:23:48:279178: schedule_coef: -0.2, tick_coef: 1.0
Average stock spread for sample: 3.1896
2020-03-21 18:23:48:453665: starting main loop
2020-03-21 18:26:58:087003: end simulation loop
2020-03-21 18:26:58:087115: order analytics
2020-03-21 18:26:58:187034: VWAP run complete.
-0.6309819803865366
            price  shares  bar
trade_type                    
a             355     355  355
p             481     481  481
2020-03-21 18:26:58:293052: Beginning VWAP run: s 300000 shares
2020-03-21 18:26:58:293106: schedule_coef: -0.19, tick_coef: 1.0
Average stock spread for sample: 3.1896
2020-03-21 18:26:58:483283: starting main loop
2020-03-21 18:29:52:817078: end simulation loop
2020-03-21 18:29:52:817191: order analytics
2020-03-21 18:29:52:923463: VWAP run complete.
-0.6304510193469923
            price  shares  bar
trade_type                    
a             355     355  355
p             481     481  481
202

2020-03-21 19:10:55:407396: starting main loop
2020-03-21 19:13:54:536475: end simulation loop
2020-03-21 19:13:54:536680: order analytics
2020-03-21 19:13:54:637675: VWAP run complete.
-0.6548982737378992
            price  shares  bar
trade_type                    
a             346     346  346
p             775     775  775
2020-03-21 19:13:54:734572: Beginning VWAP run: s 300000 shares
2020-03-21 19:13:54:734623: schedule_coef: -0.03, tick_coef: 1.0
Average stock spread for sample: 3.1896
2020-03-21 19:13:54:916039: starting main loop
2020-03-21 19:16:52:861961: end simulation loop
2020-03-21 19:16:52:862061: order analytics
2020-03-21 19:16:52:960814: VWAP run complete.
-0.654924066000035
            price  shares  bar
trade_type                    
a             345     345  345
p             790     790  790
2020-03-21 19:16:53:057260: Beginning VWAP run: s 300000 shares
2020-03-21 19:16:53:057312: schedule_coef: -0.02, tick_coef: 1.0
Average stock spread for sample: 3.1896
202

            price  shares   bar
trade_type                     
a             154     154   154
p            2979    2979  2979


## pick the best tick

In [9]:
schedule_coef = 0.08
tick_coefs = np.linspace(0.01, 1, 20)[::-1]
if order_side == 's':
    tick_coefs = -tick_coefs
for i, tick_coef in enumerate(tick_coefs):
    results = vw.algo_loop(day, order_side, order_quantity, vwap_coefs, schedule_coef, tick_coef)
    print(results['day_vwap'] - results['avg_price'])
    print(results['trades'].groupby(['trade_type']).count())

2020-03-18 21:40:29:766466: Beginning VWAP run: s 300000 shares
2020-03-18 21:40:29:766695: schedule_coef: 0.08, tick_coef: -1.0
Average stock spread for sample: 3.1896
2020-03-18 21:40:30:030696: starting main loop
2020-03-18 21:43:51:924275: end simulation loop
2020-03-18 21:43:51:924373: order analytics
2020-03-18 21:43:52:033629: VWAP run complete.
-0.5903547860291383
            price  shares  bar
trade_type                    
a             376     376  376
p             304     304  304
2020-03-18 21:43:52:153125: Beginning VWAP run: s 300000 shares
2020-03-18 21:43:52:153187: schedule_coef: 0.08, tick_coef: -0.9478947368421053
Average stock spread for sample: 3.1896
2020-03-18 21:43:52:351705: starting main loop


KeyboardInterrupt: 